In [2]:
#Libraries for dataframes 
import pandas as pd 
import numpy as np 

#Libraries for train-test and evaluation 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Libraries for deep learning 
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import os
import shutil

In [3]:
#Loading in the images into a dataframe that holds the image name, label, and whether or not it's part of test or validation set 

categories = ["MildDemented","ModerateDemented","NonDemented","VeryMildDemented"]

trainDf = pd.DataFrame(columns = ["imageName","label","trainOrTest"])
testDf = pd.DataFrame(columns = ["imageName","label","trainOrTest"])

train_main_folder = r"train"
train_subfolders = [f.path for f in os.scandir(train_main_folder) if f.is_dir()]

test_main_folder = r"val"
test_subfolders = [f.path for f in os.scandir(test_main_folder) if f.is_dir()]

for folder in os.listdir("train"):
    for category in categories:
        if category == folder:
            for image in os.listdir("train\\" + str(folder)):
                row = ["train\\" + folder + "\\" + image,category,"train"]
                trainDf = pd.concat([pd.DataFrame([row], columns=trainDf.columns), trainDf], ignore_index=True)


for folder in os.listdir("val"):
    for category in categories:
        if category == folder:
            for image in os.listdir("val\\" + str(folder)):
                row = ["val\\" + folder + "\\" + image,category,"test"]
                testDf = pd.concat([pd.DataFrame([row], columns=testDf.columns), testDf], ignore_index=True)             

In [ ]:
trainDf

In [4]:
testDf

,imageName,label,trainOrTest
0,val\VeryMildDemented\verymildDem999.jpg,VeryMildDemented,test
1,val\VeryMildDemented\verymildDem998.jpg,VeryMildDemented,test
2,val\VeryMildDemented\verymildDem997.jpg,VeryMildDemented,test
3,val\VeryMildDemented\verymildDem996.jpg,VeryMildDemented,test
4,val\VeryMildDemented\verymildDem995.jpg,VeryMildDemented,test
...,...,...,...
6395,val\MildDemented\26 (23).jpg,MildDemented,test
6396,val\MildDemented\26 (22).jpg,MildDemented,test
6397,val\MildDemented\26 (21).jpg,MildDemented,test
6398,val\MildDemented\26 (20).jpg,MildDemented,test


In [14]:
imageWidth = 200 
imageHeight = 190
batchSize = 512


train_data_generator = ImageDataGenerator(rescale = 1.0/255,
                                          zoom_range = .2,
                                          fill_mode = 'nearest',
                                          horizontal_flip = True,
                                          vertical_flip = True,
                                          brightness_range=[0.4,1.5]
)

train_data_generator2 = train_data_generator.flow_from_directory("train",
                                                                 target_size = (imageWidth, imageHeight),
                                                                 class_mode = 'categorical',
                                                                 batch_size = batchSize)

Found 33984 images belonging to 4 classes.


In [15]:
test_data_generator = ImageDataGenerator(rescale = 1.0/255)

test_data_generator2 = test_data_generator.flow_from_directory("val",
                                                               target_size = (imageWidth,imageHeight),
                                                               class_mode = 'categorical',
                                                               batch_size = batchSize)

Found 6400 images belonging to 4 classes.


In [16]:
print(train_data_generator2.class_indices.items())
label_mappings = {}
for key,value in train_data_generator2.class_indices.items():
    label_mappings[key] = value 
print(label_mappings)

dict_items([('MildDemented', 0), ('ModerateDemented', 1), ('NonDemented', 2), ('VeryMildDemented', 3)])
{'MildDemented': 0, 'ModerateDemented': 1, 'NonDemented': 2, 'VeryMildDemented': 3}


In [17]:
def create_model(): 
    model = Sequential([Conv2D(filters=128,kernel_size=(3,3),padding = "valid",input_shape=(imageWidth,imageHeight,3)),
                        Activation('relu'),
                        MaxPooling2D(pool_size = (2,2)),

                        Conv2D(filters=64,kernel_size=(3,3),padding="valid"),
                        Activation('relu'),
                        MaxPooling2D(pool_size=(2,2)),

                        Conv2D(filters=32,kernel_size=(3,3),padding="valid"),
                        Activation('relu'),
                        MaxPooling2D(pool_size=(2,2)),

                        Flatten(),

                        Dense(units=4,activation='relu'),
                        Dropout(.5),
                        Dense(units=4,activation='softmax')
                        ])
    return model


In [18]:
cnn_model = create_model()
print(cnn_model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 188, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 198, 188, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 94, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 97, 92, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 97, 92, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 44, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 46, 44, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 16192)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │        64,772 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,632 (627.47 KB)

 Trainable params: 160,632 (627.47 KB)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
optimizer = Adam(learning_rate=.002)

In [20]:
cnn_model.compile(optimizer=optimizer,loss='mse',metrics=['accuracy'])

In [21]:
hist = cnn_model.fit(train_data_generator2,epochs = 25,validation_data = test_data_generator2,
                    verbose=2)

Epoch 1/25
